
# パラメータ管理

アーキテクチャを選択してハイパーパラメータを設定したら、トレーニング ループに進みます。ここでの目標は、損失関数を最小化するパラメータ値を見つけることです。トレーニング後、将来の予測を行うためにこれらのパラメーターが必要になります。さらに、他のコンテキストでパラメータを再利用したり、他のソフトウェアで実行できるようにモデルをディスクに保存したり、科学的な理解を得るために検査するためにパラメータを抽出したい場合もあります。

ほとんどの場合、パラメータの宣言と操作の詳細については無視して、ディープ ラーニング フレームワークに頼って面倒な作業を行うことができます。ただし、標準レイヤーを備えたスタック型アーキテクチャから離れると、パラメーターの宣言と操作という雑草に手を染める必要が生じることがあります。このセクションでは、次の内容について説明します。
- デバッグ、診断、視覚化のためのパラメータへのアクセス。
- 異なるモデルコンポーネント間でパラメータを共有します。


In [1]:
import torch
from torch import nn


(**まず、1 つの隠れ層を持つ MLP に焦点を当てます。** )


In [2]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])


## [**パラメータアクセス**]

 :label: `subsec_param-access`

すでに知っているモデルからパラメーターにアクセスする方法から始めましょう。



モデルが`Sequential`クラスを介して定義されている場合、リストであるかのようにモデルにインデックスを付けることで、最初に任意のレイヤーにアクセスできます。各レイヤーのパラメーターは、その属性に簡単に配置できます。



次のようにして、2 番目の全結合層のパラメーターを検査できます。


In [3]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.2523,  0.2104,  0.2189, -0.0395, -0.0590,  0.3360, -0.0205, -0.1507]])),
             ('bias', tensor([0.0694]))])


この全結合層には、その層の重みとバイアスにそれぞれ対応する 2 つのパラメーターが含まれていることがわかります。

###  【**対象パラメータ**】

各パラメータはパラメータ クラスのインスタンスとして表されることに注意してください。パラメータを使って何か役立つことをするには、まず基礎となる数値にアクセスする必要があります。これを行うにはいくつかの方法があります。より単純なものもあれば、より一般的なものもあります。次のコードは、パラメーター クラスのインスタンスを返し、さらにそのパラメーターの値にアクセスする 2 番目のニューラル ネットワーク層からバイアスを抽出します。


In [4]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.0694]))


パラメータは、値、勾配、追加情報を含む複雑なオブジェクトです。このため、値を明示的にリクエストする必要があります。

値に加えて、各パラメーターを使用して勾配にアクセスすることもできます。このネットワークに対してバックプロパゲーションをまだ呼び出していないため、初期状態にあります。


In [5]:
net[2].weight.grad == None

True


### [**すべてのパラメータを一度に**]

すべてのパラメータに対して操作を実行する必要がある場合、パラメータに 1 つずつアクセスするのは面倒になる可能性があります。より複雑なモジュール (例: ネストされたモジュール) を扱う場合、各サブモジュールのパラメータを抽出するためにツリー全体を再帰する必要があるため、状況は特に扱いにくくなる可能性があります。以下では、すべてのレイヤーのパラメーターにアクセスする方法を示します。


In [6]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]


## [**関連付けられたパラメータ**]

多くの場合、複数のレイヤー間でパラメータを共有したいことがあります。これをエレガントに行う方法を見てみましょう。以下では、完全に接続されたレイヤーを割り当て、そのパラメーターを特に使用して別のレイヤーのパラメーターを設定します。ここでは、パラメータにアクセスする前に順伝播`net(X)`を実行する必要があります。


In [7]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])



この例は、2 番目と 3 番目のレイヤーのパラメーターが関連付けられていることを示しています。それらは単に等しいだけではなく、まったく同じテンソルで表されます。したがって、パラメータの 1 つを変更すると、他のパラメータも変更されます。



パラメーターが関連付けられると、グラデーションはどうなるのかと疑問に思うかもしれません。モデル パラメーターには勾配が含まれているため、2 番目の隠れ層と 3 番目の隠れ層の勾配はバックプロパゲーション中に加算されます。



## まとめ

モデルパラメータにアクセスして結び付ける方法はいくつかあります。

## 演習
1. :numref: `sec_model_construction`で定義された`NestMLP`モデルを使用し、さまざまなレイヤーのパラメーターにアクセスします。
1. 共有パラメータ層を含む MLP を構築し、トレーニングします。トレーニング プロセス中に、各レイヤーのモデル パラメーターと勾配を観察します。
1. パラメーターを共有することがなぜ良い考えなのでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/57)
